In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"],1)
application_df.head()

C:\Users\tamik\AppData\Local\Temp\ipykernel_20232\1309730736.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN", "NAME"],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count =application_df["APPLICATION_TYPE"].value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_count[application_type_count<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_type_count =application_df["CLASSIFICATION"].value_counts()
classification_type_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_type_count[classification_type_count<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Define the build_model function with hyperparameters
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(43,)))
    
    # Search for the best activation function
    activation = hp.Choice('activation', values=['relu', 'tanh'])
        
    # Search for the number of neurons in the first layer
    num_neurons = hp.Int('num_neurons', min_value=1, max_value=30, step=5)

    # Define the number of hidden layers as a hyperparameter
    num_hidden_layers = hp.Int('num_hidden_layers', min_value=1, max_value=5, default=2)
    
    # Define the number of units in each hidden layer as a hyperparameter
    num_units = hp.Int('num_units', min_value=1, max_value=30, default=16)
    
    for i in range(num_hidden_layers):
        model.add(layers.Dense(num_units, activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return model

# Create a Keras Tuner RandomSearch tuner
tuner = RandomSearch(
    build_model,           
    objective='val_accuracy',
    max_trials=5,        
    directory='tuner_dir_6', 
    project_name='AlphabetSoup_5'
)

# Run the tuner search with data (X_train_scaled, y_train, X_test_scaled, y_test)
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))


Trial 5 Complete [00h 00m 35s]
val_accuracy: 0.7294460535049438

Best val_accuracy So Far: 0.7336443066596985
Total elapsed time: 00h 09m 03s


In [13]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'num_neurons': 26, 'num_hidden_layers': 5, 'num_units': 15}
{'activation': 'tanh', 'num_neurons': 16, 'num_hidden_layers': 1, 'num_units': 16}
{'activation': 'relu', 'num_neurons': 21, 'num_hidden_layers': 1, 'num_units': 23}


In [14]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'num_neurons': 26,
 'num_hidden_layers': 5,
 'num_units': 15}

In [15]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5564 - accuracy: 0.7336 - 520ms/epoch - 2ms/step
Loss: 0.556403398513794, Accuracy: 0.7336443066596985


In [16]:
#save best model 
best_model.save('bestmodel_2.h5')

C:\Users\tamik\anaconda3\envs\bootcamp3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  26
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 15
hidden_nodes_layer4 = 15
hidden_nodes_layer5 = 15


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fith hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 26)                1144      
                                                                 
 dense_7 (Dense)             (None, 15)                405       
                                                                 
 dense_8 (Dense)             (None, 15)                240       
                                                                 
 dense_9 (Dense)             (None, 15)                240       
                                                                 
 dense_10 (Dense)            (None, 15)                240       
                                                                 
 dense_11 (Dense)            (None, 1)                 16        
                                                                 
Total params: 2285 (8.93 KB)
Trainable params: 2285 (8

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights_opt_2.h5",  
    save_weights_only=True,       
    save_best_only=False,         
    save_freq=5)   

fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5842 - accuracy: 0.7031
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5546 - accuracy: 0.7281
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5510 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5502 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5482 - accuracy: 0.7323
Epoch 6/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5480 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5469 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5461 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5456 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5453 - accura

804/804 [==============================] - 4s 5ms/step - loss: 0.5353 - accuracy: 0.7394
Epoch 82/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5351 - accuracy: 0.7399
Epoch 83/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5352 - accuracy: 0.7397
Epoch 84/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5350 - accuracy: 0.7393
Epoch 85/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5350 - accuracy: 0.7388
Epoch 86/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5347 - accuracy: 0.7400
Epoch 87/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5348 - accuracy: 0.7393
Epoch 88/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5348 - accuracy: 0.7388
Epoch 89/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5347 - accuracy: 0.7397
Epoch 90/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5352 - accuracy: 

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5571 - accuracy: 0.7310 - 453ms/epoch - 2ms/step
Loss: 0.5570513010025024, Accuracy: 0.7309620976448059


In [25]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_best_opt.h5')

In [21]:
# Train the model with more epochs
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights_opt_3.h5",  
    save_weights_only=True,       
    save_best_only=False,         
    save_freq=5)   

fit_model_2 = nn.fit(X_train_scaled,y_train,epochs=150, callbacks=[checkpoint_callback])

Epoch 1/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5341 - accuracy: 0.7407
Epoch 2/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5347 - accuracy: 0.7404
Epoch 3/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5345 - accuracy: 0.7400
Epoch 4/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5343 - accuracy: 0.7401
Epoch 5/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5343 - accuracy: 0.7397
Epoch 6/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5339 - accuracy: 0.7403
Epoch 7/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5343 - accuracy: 0.7391
Epoch 8/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5340 - accuracy: 0.7399
Epoch 9/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5340 - accuracy: 0.7393
Epoch 10/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5343 - accura

804/804 [==============================] - 4s 5ms/step - loss: 0.5321 - accuracy: 0.7413
Epoch 82/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5322 - accuracy: 0.7406
Epoch 83/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5321 - accuracy: 0.7406
Epoch 84/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5319 - accuracy: 0.7415
Epoch 85/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5330 - accuracy: 0.7413
Epoch 86/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5320 - accuracy: 0.7414
Epoch 87/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5321 - accuracy: 0.7402
Epoch 88/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5320 - accuracy: 0.7406
Epoch 89/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5321 - accuracy: 0.7400
Epoch 90/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5320 - accuracy: 

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5809 - accuracy: 0.7307 - 455ms/epoch - 2ms/step
Loss: 0.5809048414230347, Accuracy: 0.7307288646697998


In [23]:
# Train the model with more epochs
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights_opt_4.h5",  
    save_weights_only=True,       
    save_best_only=False,         
    save_freq=5)   

fit_model_2 = nn.fit(X_train_scaled,y_train,epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5316 - accuracy: 0.7409
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5313 - accuracy: 0.7402
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5312 - accuracy: 0.7406
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5314 - accuracy: 0.7418
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5315 - accuracy: 0.7412
Epoch 6/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5315 - accuracy: 0.7413
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5312 - accuracy: 0.7417
Epoch 8/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5314 - accuracy: 0.7411
Epoch 9/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5314 - accuracy: 0.7416
Epoch 10/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5311 - accuracy: 0.7414

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5807 - accuracy: 0.7275 - 369ms/epoch - 1ms/step
Loss: 0.5807419419288635, Accuracy: 0.7274635434150696
